![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>Model Editing  <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/mjspec.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

This notebook provides an introductory tutorial for model editing in MuJoCo using the `mjSpec` API. This notebook assumes that the reader is already familiar with MuJoCo basic concepts, as demostrated in the [introductory tutorial](https://github.com/google-deepmind/mujoco?tab=readme-ov-file#getting-started). Documentation for this API can be found in the [Model Editing](https://mujoco.readthedocs.io/en/latest/programming/modeledit.html) chapter in the documentation (C API) and in the [Python chapter](https://mujoco.readthedocs.io/en/latest/python.html#model-editing). Here we use the Python API.

The goal of the API is to allow users to easily interact with and modify MuJoCo
models in Python, similarly to what the JavaScript DOM does for HTML.

<!-- Copyright 2024 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->


## All imports

In [ ]:
!pip install mujoco

# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco as mj
  mj.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np
import typing
import dataclasses
import random
import math

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

from pygments import highlight
from pygments.lexers import XmlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML, display

def print_xml(xml_string):
    formatter = HtmlFormatter(style='lovelace')
    highlighted_xml = highlight(xml_string, XmlLexer(), formatter)
    display(HTML(f"<style>{formatter.get_style_defs()}</style>{highlighted_xml}"))

import matplotlib.pyplot as plt
import matplotlib.animation as animation

def render(model, data=None, height=250):
  if data is None:
    data = mj.MjData(model)
  with mj.Renderer(model, 480, 640) as renderer:
    mj.mj_forward(model, data)
    renderer.update_scene(data)
    media.show_image(renderer.render(), height=height)

class Arena:
  def __init__(self):
    self.spec = mj.MjSpec()
    self.model = self.spec.worldbody
    self.spec.compiler.degree = False
    self.scene_option = mj.MjvOption()

    # Make arena with textured floor.
    chequered = self.spec.add_texture(
        name="chequered", type=mj.mjtTexture.mjTEXTURE_2D,
        builtin=mj.mjtBuiltin.mjBUILTIN_CHECKER,
        width=300, height=300, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])
    grid = self.spec.add_material(
        name='grid', texrepeat=[5, 5], reflectance=.2
        ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'chequered'
    self.model.add_geom(
        type=mj.mjtGeom.mjGEOM_PLANE, size=[2, 2, .1], material='grid')
    for x in [-2, 2]:
      self.model.add_light(pos=[x, -1, 3], dir=[-x, 1, -2])

  def add_tracking_camera(self,body,name="eye",pos=[0,0,1],euler=[0,0,0]):
    # camera
    self.model.add_camera(name=name,
                          mode =mj.mjtCamLight.mjCAMLIGHT_TRACK ,
                          targetbody=body,
                          pos= pos,
                          euler=euler)
    return name

  def visulize_camera(self):
    self.scene_option.flags[mj.mjtVisFlag.mjVIS_CAMERA] = True

  def add_movable_asset(self,body,pos=[0,0,0],prefix='_'):
    frame = self.model.add_frame(pos=pos)
    body = frame.attach_body(body,  prefix,'')
    body.add_freejoint()
    return body

  def add_fixed_asset(self,body,pos=[0,0,0],prefix='_'):
    frame = self.model.add_frame(pos=pos)
    body = frame.attach_body(body,  prefix,'')
    return body

  def remove_asset(self,body):
    self.spec.detach_body(body)


def render_video(robot,movable = False, width = 320, height = 200):
  arena = Arena()
  if movable:
    arena.add_movable_asset(robot_root,[0,0,0.11])
  else:
    arena.add_fixed_asset(robot_root,[0,0,0.11])

  model = arena.spec.compile()
  data = mj.MjData(model)
  random_state = np.random.RandomState(42)

  actuators = [data.bind(actuator) for actuator in arena.spec.actuators]

  # Control signal frequency, phase, amplitude.
  freq = 5
  phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
  amp = 0.9

  # Simulate and display video.
  duration = 3    # (seconds)
  framerate = 30  # (Hz)
  frames = []
  mj.mj_resetData(model, data)
  with mj.Renderer(model,width= width ,height= height) as renderer:
    while data.time < duration:
      # Inject controls and step the physics.
      for i, actuator in enumerate(actuators):
        actuator.ctrl = amp * np.sin(freq * data.time + phase[i])
      mj.mj_step(model, data)
      if len(frames) < data.time * framerate:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)

  media.show_video(frames, fps=framerate)

# Parsing XML to `mjSpec` and compiling to `mjModel`


Unlike `mj_loadXML` which combines parsing and compiling, when using `mjSpec`, parsing and compiling are separate, allowing for editing steps:

In [ ]:
#@title A static model, from string {vertical-output: true}

static_model = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
spec = mj.MjSpec.from_string(static_model)
model = spec.compile()
render(model)

# Change the mjSpec, re-compile and re-render
geoms = spec.worldbody.find_all(mj.mjtObj.mjOBJ_GEOM)
geoms[0].name = 'blue_box'
geoms[0].rgba = [0, 0, 1, 1]
geoms[1].name = 'yellow_sphere'
geoms[1].rgba = [1, 1, 0, 1]
spec.worldbody.add_geom(name='magenta cylinder',
                        type=mj.mjtGeom.mjGEOM_CYLINDER,
                        rgba=[1, 0, 1, 1],
                        pos=[-.2, 0, .2],
                        size=[.1, .1, 0])

model = spec.compile()
render(model)

`mjSpec` can save XML to string, saving all modifications.

In [ ]:
#@title Print an XML from an `mjSpec` {vertical-output: true}

print_xml(spec.to_xml())

In [ ]:
#@title Building an `mjSpec` from scratch {vertical-output: true}

spec = mj.MjSpec()
spec.worldbody.add_light(name="top", pos=[0, 0, 1])
body = spec.worldbody.add_body(name="box_and_sphere", euler=[0, 0, -30])
body.add_joint(name="swing", type=mj.mjtJoint.mjJNT_HINGE,
               axis=[1, -1, 0], pos=[-.2, -.2, -.2])
body.add_geom(name="red_box", type=mj.mjtGeom.mjGEOM_BOX,
              size=[.2, .2, .2], rgba=[1, 0, 0, 1])
body.add_geom(name="green_sphere", pos=[.2, .2, .2],
              size=[.1, 0, 0], rgba=[0, 1, 0, 1])
model = spec.compile()

duration = 2    # (seconds)
framerate = 30  # (Hz)

# enable joint visualization option:
scene_option = mj.MjvOption()
scene_option.flags[mj.mjtVisFlag.mjVIS_JOINT] = True

# Simulate and display video.
frames = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)


# Control example

A key feature of this library is the ability to easily attach multiple models into a larger one. Disambiguation of duplicated names from different
models, or multiple instances of the same model, is handled via user-defined namespacing.

One example use case is when we want robots with a variable number of joints, as is a fundamental change to the kinematic structure. The following snippets realise this scenario.

In [ ]:
leg_model = """
<mujoco>
  <compiler angle="radian"/>

  <default>
    <joint damping="2" type="hinge"/>
    <geom type="capsule"/>
  </default>

  <worldbody>
    <body name="thigh">
      <joint name="hip" axis="0 0 1"/>
      <body name="shin">
        <joint name="knee" axis="0 1 0"/>
      </body>
    </body>
  </worldbody>

  <actuator>
    <position joint="hip" kp="10"/>
    <position joint="knee" kp="10"/>
  </actuator>
</mujoco>
"""

class Leg(object):
  """A 2-DoF leg with position actuators."""
  def __init__(self, length, rgba):
    self.spec = mj.MjSpec.from_string(leg_model)

    # Thigh:
    thigh = self.spec.find_body('thigh')
    thigh.add_geom(fromto=[0, 0, 0, length, 0, 0], size=[length/4, 0, 0], rgba=rgba)

    # Hip:
    shin = self.spec.find_body('shin')
    shin.add_geom(fromto=[0, 0, 0, 0, 0, -length], size=[length/5, 0, 0], rgba=rgba)
    shin.pos[0] = length

The `Leg` class describes an abstract articulated leg, with two joints and corresponding proportional-derivative actuators.

Note that:

- MJCF attributes correspond directly to arguments of the `add_()` methods.
- When referencing elements, e.g when specifying the joint to which an actuator is attached, the name string of the MJCF elements is used.

In [ ]:
BODY_RADIUS = 0.1
random_state = np.random.RandomState(42)
creature_model = """
<mujoco>
  <compiler angle="radian"/>

  <worldbody>
    <geom name="torso" type="ellipsoid" size="{} {} {}"/>
  </worldbody>
</mujoco>
""".format(BODY_RADIUS, BODY_RADIUS, BODY_RADIUS / 2)

def make_creature(num_legs):
  """Constructs a creature with `num_legs` legs."""
  rgba = random_state.uniform([0, 0, 0, 1], [1, 1, 1, 1])
  spec = mj.MjSpec.from_string(creature_model)

  # Attach legs to equidistant sites on the circumference.
  spec.worldbody.first_geom().rgba = rgba
  leg = Leg(length=BODY_RADIUS, rgba=rgba)
  for i in range(num_legs):
    theta = 2 * i * np.pi / num_legs
    hip_pos = BODY_RADIUS * np.array([np.cos(theta), np.sin(theta), 0])
    hip_site = spec.worldbody.add_site(pos=hip_pos, euler=[0, 0, theta])
    hip_site.attach(leg.spec.find_body('thigh'), '', '-' + str(i))

  return spec

The `make_creature` function uses the `attach()` method to procedurally attach legs to the torso. Note that at this stage both the torso and hip attachment sites are children of the `worldbody`, since their parent body has yet to be instantiated. We'll now make an arena with a chequered floor and two lights, and place our creatures in a grid.

In [ ]:
#@title Six Creatures on a floor.{vertical-output: true}

arena = mj.MjSpec()

if hasattr(arena, 'compiler'):
  arena.compiler.degree = False  # MuJoCo dev (next release).
else:
  arena.degree = False  # MuJoCo release

# Make arena with textured floor.
chequered = arena.add_texture(
    name="chequered", type=mj.mjtTexture.mjTEXTURE_2D,
    builtin=mj.mjtBuiltin.mjBUILTIN_CHECKER,
    width=300, height=300, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])
grid = arena.add_material(
    name='grid', texrepeat=[5, 5], reflectance=.2
    ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'chequered'
arena.worldbody.add_geom(
    type=mj.mjtGeom.mjGEOM_PLANE, size=[2, 2, .1], material='grid')
for x in [-2, 2]:
  arena.worldbody.add_light(pos=[x, -1, 3], dir=[-x, 1, -2])

# Instantiate 6 creatures with 3 to 8 legs.
creatures = [make_creature(num_legs=num_legs) for num_legs in range(3, 9)]

# Place them on a grid in the arena.
height = .15
grid = 5 * BODY_RADIUS
xpos, ypos, zpos = np.meshgrid([-grid, 0, grid], [0, grid], [height])
for i, spec in enumerate(creatures):
  # Place spawn sites on a grid.
  spawn_pos = (xpos.flat[i], ypos.flat[i], zpos.flat[i])
  spawn_site = arena.worldbody.add_site(pos=spawn_pos, group=3)
  # Attach to the arena at the spawn sites, with a free joint.
  spawn_body = spawn_site.attach(spec.worldbody, '', '-' + str(i))
  spawn_body.add_freejoint()

# Instantiate the physics and render.
model = arena.compile()
render(model)

Multi-legged creatures, ready to roam! Let's inject some controls and watch them move. We'll generate a sinusoidal open-loop control signal of fixed frequency and random phase, recording both video frames and the horizontal positions of the torso geoms, in order to plot the movement trajectories.

In [ ]:
#@title Video of the movement{vertical-output: true}

data = mj.MjData(model)
duration = 10   # (Seconds)
framerate = 30  # (Hz)
video = []
pos_x = []
pos_y = []
geoms = arena.worldbody.find_all(mj.mjtObj.mjOBJ_GEOM)
torsos_data = [data.bind(geom) for geom in geoms if 'torso' in geom.name]
torsos_model = [model.bind(geom) for geom in geoms if 'torso' in geom.name]
actuators = [data.bind(actuator) for actuator in arena.actuators]

# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.actuators))
amp = 0.9

# Simulate, saving video frames and torso locations.
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    for i, actuator in enumerate(actuators):
      actuator.ctrl = amp * np.sin(freq * data.time + phase[i])
    mj.mj_step(model, data)

    # Save torso horizontal positions using name indexing.
    pos_x.append([torso.xpos[0] for torso in torsos_data])
    pos_y.append([torso.xpos[1] for torso in torsos_data])

    # Save video frames.
    if len(video) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      video.append(pixels.copy())

media.show_video(video, fps=framerate)

In [ ]:
#@title Movement trajectories{vertical-output: true}

creature_colors = model.geom_rgba[torsos][:, :3]
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_prop_cycle(color=creature_colors)
_ = ax.plot(pos_x, pos_y, linewidth=4)

The plot above shows the corresponding movement trajectories of creature positions. Note how `mjSpec` attribute `id` were used to access both `xpos` and `rgba` values. This attribute is valid only after a model is compiled.

# Model editing

First we need to download models from mujoco_menagerie

In [ ]:
# Get MuJoCo's humanoid model and a Franka arm from the MuJoCo Menagerie.
print('Getting MuJoCo humanoid XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco
humanoid_file = 'mujoco/model/humanoid/humanoid.xml'
humanoid100_file = 'mujoco/model/humanoid/humanoid100.xml'
print('Getting MuJoCo Menagerie Franka XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco_menagerie
franka_file = 'mujoco_menagerie/franka_fr3/fr3.xml'

`mjSpec` elements can be traversed in two ways:
- For all other elements, we provide a list. https://mujoco.readthedocs.io/en/latest/python.html#element-lists
- For elements inside the kinematic tree, the tree can be traversed using the `first` and `next` functions.




In [ ]:
#@title Traversing the spec.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

print("The spec has the following actuators:")
for actuator in spec.actuators:
  print(actuator.name)

# Function that recursively prints all body names
def print_bodies(parent, level=0):
  body = parent.first_body()
  while body:
    print(''.join(['-' for i in range(level)]) + body.name)
    print_bodies(body, level + 1)
    body = parent.next_body(body)

print("\nThe spec has the following bodies:")
print_bodies(spec.worldbody)

An `mjSpec` can be compiled multiple times. If the state has to be preserved between different compilations, then the function `recompile()` must be used, which returns a new `mjData` that contains the mapped state, possibly having a different dimension from the origin.

In [ ]:
#@title Model re-compilation with state preservation.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid100_file)
model = spec.compile()
data = mj.MjData(model)

# Run for 5 seconds
for i in range(1000):
  mj.mj_step(model, data)

# Show result
render(model, data)

# Create list of all bodies we want to delete
body = spec.worldbody.first_body()
delete_list = []
while body:
  geom_type = body.first_geom().type
  if (geom_type == mj.mjtGeom.mjGEOM_BOX or
      geom_type == mj.mjtGeom.mjGEOM_ELLIPSOID):
    delete_list.append(body)
  body = spec.worldbody.next_body(body)

# Remove all bodies in the list from the spec
for body in delete_list:
  spec.detach_body(body)

# # Add another humanoid
spec_humanoid = mj.MjSpec.from_file(humanoid_file)
attachment_frame = spec.worldbody.add_frame(pos=[0, -1, 1])
attachment_frame.attach_body(spec_humanoid.find_body('torso'), 'a', 'b')

# Recompile preserving the state
new_model, new_data = spec.recompile(model, data)

# Show result
render(new_model, new_data)

Let us load the humanoid model and inspect it.

In [ ]:
#@title Humanoid model.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

model = spec.compile()
render(model)

We wish to remove the arms and replace them with the legs. This can be done by first storing the arm positions into frames attaches to the torso. Then we can detach the arms and self-attach the legs into the frames.

In [ ]:
#@title Humanoid with arms replaced by legs.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

# Get the torso, arm, and leg bodies
arm_left = spec.find_body('upper_arm_left')
arm_right = spec.find_body('upper_arm_right')
leg_left = spec.find_body('thigh_left')
leg_right = spec.find_body('thigh_right')
torso = spec.find_body('torso')

# Attach frames at the arm positions
shoulder_left = torso.add_frame(pos=arm_left.pos)
shoulder_right = torso.add_frame(pos=arm_right.pos)

# Remove the arms
spec.detach_body(arm_left)
spec.detach_body(arm_right)

# Add new legs
shoulder_left.attach_body(leg_left, 'shoulder', 'left')
shoulder_right.attach_body(leg_right, 'shoulder', 'right')

model = spec.compile()
render(model, height=400)

Similarly, different models can be attach together. Here, the right arm is detached and a robot arm from a different model is attached in its place.

In [ ]:
#@title Humanoid with Franka arm.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)
franka = mj.MjSpec.from_file(franka_file)

if hasattr(spec, 'compiler'):
  spec.compiler.degree = False  # MuJoCo dev (next release).
else:
  spec.degree = False  # MuJoCo release

# Replace right arm with frame
arm_right = spec.find_body('upper_arm_right')
torso = spec.find_body('torso')
shoulder_right = torso.add_frame(pos=arm_right.pos, quat=[0, 0.8509035, 0, 0.525322])
spec.detach_body(arm_right)

# Attach Franka arm to humanoid
franka_arm = franka.find_body('fr3_link2')
shoulder_right.attach_body(franka_arm, 'franka', '')

model = spec.compile()
render(model, height=400)

When doing this, the actuators and all other objects referenced by the attached sub-tree are imported in the new model. All assets are currently imported, referenced or not.

In [ ]:
#@title Imported actuators.{vertical-output: true}

for actuator in spec.actuators:
  print(actuator.name)

Domain randomization can be performed by attaching multiple times the same spec, edited each time with a new instance of randomized parameters.

In [ ]:
#@title Humanoid with randomized heads and arm poses.{vertical-output: true}

humanoid = mj.MjSpec.from_file(humanoid_file)
spec = mj.MjSpec()

# Delete all key frames to avoid name conflicts
while humanoid.keys:
  humanoid.keys[-1].delete()

# Create a grid of humanoids by attaching humanoid to spec multiple times
for i in range(4):
  for j in range(4):
    humanoid.materials[0].rgba = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), 1]  # Randomize color
    humanoid.find_body('head').first_geom().size = [
        .18*np.random.uniform(), 0, 0]  # Randomize head size
    humanoid.find_body('upper_arm_left').quat = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), np.random.uniform()]  # Randomize left arm orientation
    humanoid.find_body('upper_arm_right').quat = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), np.random.uniform()]  # Randomize right arm orientation

    # attach randomized humanoid to parent spec
    frame = spec.worldbody.add_frame(pos=[i, j, 0])
    frame.attach_body(humanoid.find_body('torso'), str(i), str(j))

spec.worldbody.add_light(mode=mj.mjtCamLight.mjCAMLIGHT_TARGETBODYCOM,
                         targetbody='3torso3', diffuse=[.8, .8, .8],
                         specular=[0.3, 0.3, 0.3], pos=[0, -6, 4], cutoff=30)
model = spec.compile()
render(model, height=400)

# Pure Models with `mjspec`

A robot operates within an environment. We refer to this environment as the Arena. To add versatility to the environment, we should be able to add or remove things from the environment. To enable these functionalities in the Arena, we will use mjspec's attach, detach, and frame functionalities.



## Creating a Robot
Next, we will look at how to assemble a robot. We are going to create a quadruped with four identical legs. The leg model has three parts: the robot tree structure, actuation, and sensors.

In [ ]:
def leg_model():
  spec = mj.MjSpec()
  spec.compiler.degree = False

  spec.add_material(name = "yellow", rgba = [1,1,0,1])

  # defaults for joint and geom
  main = spec.default()
  main.joint.damping = 10
  main.geom.material = "yellow"
  main.geom.type = mj.mjtGeom.mjGEOM_CAPSULE
  main.geom.mass = 1

  root = spec.worldbody
  body = root
  angle, length, radius = 0, 0.1, 0.03
  fromto = [
    [0]*3 + [length] + [0]*2,
    [0]*5 + [-length]
  ]
  j_axis = [[0, 0, 1],[0, 1, 0]]
  # tree
  for i in range(2):
    body = body.add_body(name = "l" + str(i), pos = [length*i, 0, 0])
    body.add_geom(fromto = fromto[i], size = [length/4, 0, 0])
    body.add_joint(name = "j" + str(i), axis= j_axis[i])
  body.add_site(name = "feet", pos = [0,0, -(length + radius)])

  # defaults for position actuation
  kp = 10
  main.actuator.trntype = mj.mjtTrn.mjTRN_JOINT
  main.actuator.gaintype = mj.mjtGain.mjGAIN_FIXED
  main.actuator.biastype = mj.mjtBias.mjBIAS_AFFINE
  main.actuator.gainprm[0] =  kp
  main.actuator.biasprm[1] = -kp

  # actuators
  for i in range(2):
    spec.add_actuator(name = "act" + str(i),
                       target = "j" + str(i))
  # joint sensor
  spec.add_sensor(name = "j0_sensor",
                  objname = "j0",
                  type = mj.mjtSensor.mjSENS_JOINTPOS,
                  objtype = mj.mjtObj.mjOBJ_JOINT
                )
  # force sensor
  spec.add_sensor(name = "force_sensor",
                  objname = "feet",
                  type = mj.mjtSensor.mjSENS_FORCE,
                  objtype = mj.mjtObj.mjOBJ_SITE
                )

  return root ,spec

In [ ]:
_, spec = leg_model()
model = spec.compile()
print_xml(spec.to_xml())

Lets add this model to arena, give it some motor command and look at joint sensor value.

In [ ]:
arena = Arena()
robot_root, _ = leg_model()
arena.add_fixed_asset(robot_root,[0,0,0.11])
model = arena.spec.compile()
data = mj.MjData(model)

random_state = np.random.RandomState(42)

print_xml(arena.spec.to_xml())

actuators = [data.bind(actuator) for actuator in arena.spec.actuators]
sensors = [data.bind(sensor) for sensor in arena.spec.sensors]
sensors = dict(zip(
    [sensor.name for sensor in arena.spec.sensors],
    [i for i, _ in enumerate(sensors)]
))

print(f"Sensors: {sensors}")

# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
amp = 0.9

# Simulate and display video.
duration = 3    # (seconds)
framerate = 30  # (Hz)
frames = []
hip_jpos = []

mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    for i, actuator in enumerate(actuators):
      actuator.ctrl = amp * np.sin(freq * data.time + phase[i])
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)
      hip_jpos.append(data.sensordata[sensors['_j0_sensor']])

media.show_video(frames, fps=framerate)

fig, ax = plt.subplots(figsize=(4, 2))
_ = ax.plot( range(len(hip_jpos)),hip_jpos, linewidth=1)
ax.set_ylabel('hip joint position')



## Component Reusablity
Here, we illustrate component reusability by creating a quadruped using the leg component.

In [ ]:
def quadruped_model():
  spec = mj.MjSpec()
  spec.compiler.degree = False
  spec.add_material(name = "yellow", rgba = [1, 1, 0, 1])

  root = spec.worldbody
  l_root, _ = leg_model()

  body = root.add_body(name = "torso")

  radius = 0.1
  body.add_geom(size = [radius] * 2+[radius / 2],
                material = "yellow",
                type = mj.mjtGeom.mjGEOM_ELLIPSOID
                )
  #legs
  for i in range(4):
    theta = 2 * i * np.pi / 4
    hip_pos = radius * np.array([np.cos(theta), np.sin(theta), 0])
    frame = body.add_frame(pos = hip_pos,euler = [0,0,theta])
    frame.attach_body(l_root, '', 'leg' + str(i))

  return root, spec

Lets see the robot move around!

In [ ]:
robot_root, _ = quadruped_model()
render_video(robot_root,movable = True)

## Closed Loop Kinematic and Contact Exclusion
Here, we are going to create a simplified [hexapod](https://www.youtube.com/watch?v=xiECumcaEx0) to illustrate how to use an equality constraint. The tree-like structure of the hexapod leads to closed-loop kinematics. Therefore, we need to replace two of the joints with equality constraints. We start by creating the LinearArm that actuates the hexapod.

In [ ]:
def linear_arm_model():
  spec = mj.MjSpec()
  spec.compiler.degree = False

  main = spec.default()
  main.geom.type = mj.mjtGeom.mjGEOM_CAPSULE
  main.geom.mass = 1

  kp = 200
  main.actuator.trntype = mj.mjtTrn.mjTRN_JOINT
  main.actuator.gaintype = mj.mjtGain.mjGAIN_FIXED
  main.actuator.biastype = mj.mjtBias.mjBIAS_AFFINE
  main.actuator.gainprm[0] =  kp
  main.actuator.biasprm[1] = -kp


  root = spec.worldbody
  body = root

  length, radius = 0.1, 0.03
  fromto = [[0]*5+[length], [0]*5+[length*2]]
  rgba = [[1,1,0,1], [1.0, 0.5, 0.0, 1.0]]
  j_axis = [[1,0,0],[0,0,1]]
  j_type = [mj.mjtJoint.mjJNT_HINGE, mj.mjtJoint.mjJNT_SLIDE]
  j_range = [[-1,1],[0,0.05]]
  # tree
  for i in range(2):
    body = body.add_body(name = "l"+str(i),pos = [0,0,length/2*i])
    body.add_geom(fromto = fromto[i],
                  size = [radius+(1e-4)*i,0,0],
                  rgba = rgba[i]
                  )
    body.add_joint(name = "j"+str(i),
                   axis = j_axis[i],
                   type = j_type[i],
                   range = j_range[i]
                   )
  body.add_site(name="arm_site",pos=[0,0,length*2+radius])

  #actuator
  spec.add_actuator(name="act",target="j1")

  return root, spec

Let see the robot move around! <br>
TODO: Visulization -> Activate wireframe activate joint activate activate site.

In [ ]:

robot_root, _ = linear_arm_model()
render_video(robot_root)

Here we will use LinearArm to create the Hexapod. We will illusterate Hexopod openchain, With three equality constraint and with one joint and two equality constraint. In addition we show who to exclude contact between bodies.

In [ ]:
def make_hexapod(equality3 = False,equality2 = False,
                 exclude_contact = False):
  spec = mj.MjSpec()
  spec.compiler.degree = False
  root = spec.worldbody
  arm_model ,_ = linear_arm_model()

  l_arm,r_arm = 0.1 , 0.03
  radius = 0.1

  #base
  base = root.add_body(name="base")
  base.add_geom(size=[radius]*2+[radius/2],
                type = mj.mjtGeom.mjGEOM_ELLIPSOID
                )
  arms = []
  for i in range(3):
    theta = 2 * i * np.pi / 3
    pos = radius * np.array([np.cos(theta), np.sin(theta), 0])
    frame = base.add_frame(pos = pos,euler = [0,0,theta])
    arm = frame.attach_body(arm_model,"arm" + str(i), '')
    arms.append(arm)


  top = None
  if equality3:
    # creating top panel
    top = root.add_body(name="top",pos=[0,0,(l_arm+r_arm)*2])
    top.add_geom(size=[radius]*2+[radius/2],
                type = mj.mjtGeom.mjGEOM_ELLIPSOID
                )
    # adding sites to top panel
    # and adding 3 equality constraints
    for i in range(3):
      theta = 2 * i * np.pi / 3
      site_pos = radius * np.array([np.cos(theta), np.sin(theta), 0])
      top_site = top.add_site(pos=site_pos,name="top_site"+str(i),
                              euler=[0,0,theta])
      l2 = arms[i].bodies[0].bodies[0]
      l2_site = l2.sites[0]
      spec.add_equality(objtype=mj.mjtObj.mjOBJ_SITE,
                        name1=top_site.name,
                        name2=l2_site.name
                        )

  if equality2:
    arm_0_l1 = arms[0].bodies[0].bodies[0]
    top = arm_0_l1.add_body(name="top",pos=[-radius,0,2*(l_arm)+r_arm])
    top.add_geom(size=[radius]*2+[radius/2],
                type = mj.mjtGeom.mjGEOM_ELLIPSOID
                )
    # adding one joint
    top.add_joint(
      type = mj.mjtJoint.mjJNT_BALL,
      pos = [radius,0,0]
    )
    # adding sites to top panel
    # and adding 2 equality constraints
    for i in range(1,3):
      theta = 2 * i * np.pi / 3
      site_pos = radius * np.array([np.cos(theta), np.sin(theta), 0])
      top_site = top.add_site(pos=site_pos,name="top_site"+str(i),
                              euler=[0,0,theta])
      l2 = arms[i].bodies[0].bodies[0]
      l2_site = l2.sites[0]
      spec.add_equality(objtype=mj.mjtObj.mjOBJ_SITE,
                        name1=top_site.name,
                        name2=l2_site.name
                        )

  if exclude_contact:
    for i in range(3):
      l0 = arms[i].bodies[0]
      l1 = l0.bodies[0]
      # removing contact between l0 and l1 with base
      spec.add_exclude(bodyname1=base.name,bodyname2=l0.name)
      spec.add_exclude(bodyname1=base.name,bodyname2=l1.name)
      # removing contact between l1 and top
      spec.add_exclude(bodyname1=top.name,bodyname2=l1.name)


  return root, spec, top

open chain

In [ ]:
 robot_root, _, _ = make_hexapod()
 render_video(robot_root)

3 eqaulity

In [ ]:
 robot_root, _, _ = make_hexapod(equality3 = True)
 render_video(robot_root)

2 equality and a joint

In [ ]:
 robot_root, _, _ = make_hexapod(equality2 = True)
 render_video(robot_root)

exclude collision

In [ ]:
 robot_root, _, _ = make_hexapod(equality2 = True,exclude_contact=True)
 render_video(robot_root)

## Tendons

Mujoco allows for two types of tendons: fixed and spatial tendons. We start by creating the tree-like structure of the robot. Then, we look at how to make use of fixed tendons. Finally, we make further modifications to the tree structure of the robot to define routes for spatial tendons.

In [ ]:
def arm_model_fixed_tendon():
  spec = mj.MjSpec()
  spec.compiler.degree = False
  root = spec.worldbody

  num_links = 3

  # defaults
  main = spec.default()
  main.joint.axis = [0,1,0]

  length = 0.2
  body = root
  for i in range(num_links):
    if i > 0:
      pos = [0, 0, (length+.03)]
    else:
      pos = [0]*3

    body = body.add_body(name="l"+str(i),pos = pos)
    body.add_geom(name="g1"+str(i),
                  type=mj.mjtGeom.mjGEOM_CYLINDER,
                  fromto = [0, .015, 0, 0, -.015, 0],
                  size = [.05,0,0],
                  rgba =[1.0, 0.5, 0.0, 1.0]
                  )
    body.add_geom(name= "g2"+str(i),
                  type= mj.mjtGeom.mjGEOM_CAPSULE,
                  fromto = [0]*5+[length],
                  size = [0.02,0,0],
                  rgba =[1,1,0,1]
                  )
    body.add_joint(name="j"+str(i),range=[-1,1])

  # creating tendon
  tendon = spec.add_tendon(name="tendon")
  # wrapping tendon around joints
  for i in range(num_links):
    joint = "j"+str(i)
    coef = 1
    tendon.wrap_joint(joint,coef)

  # actuation
  kp = 1
  main.actuator.gainprm[0] = kp
  main.actuator.biasprm[1] = -kp
  main.actuator.trntype = mj.mjtTrn.mjTRN_TENDON
  main.actuator.ctrlrange = [-1,1]
  spec.add_actuator(
    name="act_tendon", target="tendon"
  )

  # leaf refers to body with no parent e.g: end-effecotr
  leaf = body
  return root, leaf, spec

In [ ]:
robot_root,_, _ = arm_model_fixed_tendon()
render_video(robot_root, width = 640, height = 480)

In [ ]:
def arm_model_muscle():
  spec = mj.MjSpec()
  spec.compiler.degree = False
  root = spec.worldbody

  num_links = 3

  # defaults
  main = spec.default()
  main.joint.axis = [0,1,0]
  # muscle defaults
  main.actuator.trntype = mj.mjtTrn.mjTRN_TENDON
  main.actuator.dyntype  = mj.mjtDyn.mjDYN_MUSCLE
  main.actuator.gaintype = mj.mjtGain.mjGAIN_MUSCLE
  main.actuator.biastype = mj.mjtBias.mjBIAS_MUSCLE
  main.actuator.dynprm[0] = 0.01 # tau act
  main.actuator.dynprm[1] = 0.04 # tau deact
  # range[0], range [1], force, scale,
  # lmin, lmax, vmax, fpmax, fvmax
  main.actuator.gainprm[0:9] = [0.75,1.05,-1,200,
                                0.5,1.6,1.5,1.3,1.2]
  # biasprm = gainprm
  main.actuator.biasprm = main.actuator.gainprm[:]

  length = 0.2
  body = root
  for i in range(num_links):
    if i > 0:
      pos = [0, 0, (length+.03)]
    else:
      pos = [0]*3

    temp_b = body.add_body(name="l"+str(i),pos = pos)
    temp_b.add_geom(name="g1"+str(i),
                  type=mj.mjtGeom.mjGEOM_CYLINDER,
                  fromto = [0, .015, 0, 0, -.015, 0],
                  size = [.05,0,0],
                  rgba =[1.0, 0.5, 0.0, 1.0]
                  )
    temp_b.add_geom(name= "g2"+str(i),
                  type= mj.mjtGeom.mjGEOM_CAPSULE,
                  fromto = [0]*5+[length],
                  size = [0.02,0,0],
                  rgba =[1,1,0,1]
                  )
    temp_b.add_joint(name="j"+str(i),range=[-1,1])

    # tendon routing
    for j in range(2):
      sign = (-1)**j
      p_z = 0.15
      if i==0:
        p_z = -0.08
      # sites on parent
      body.add_site(
        name="p"+str(i)+str(j),
        pos=[sign *0.025,0,p_z]
      )
      # sidesites
      temp_b.add_site(
        name="ch_ss"+str(i)+str(j),
        pos = [sign *0.08,0,0]
      )
      # sites on child
      temp_b.add_site(
        name="ch"+str(i)+str(j),
        pos = [sign * 0.025,0,0.08]
      )
      # tendon routing
      t = spec.add_tendon(name="t"+str(i)+str(j))
      t.wrap_site("p"+str(i)+str(j))
      t.wrap_geom("g1"+str(i),"ch_ss"+str(i)+str(j))
      t.wrap_site("ch"+str(i)+str(j))
      # muscle
      spec.add_actuator(name="act"+str(i)+str(j),
                        target = "t"+str(i)+str(j)
                        )
    body = temp_b
  # leaf refers to body with no parent e.g: end-effecotr
  leaf = body
  return root,leaf, spec

In [ ]:
robot_root, _, _ = arm_model_muscle()
render_video(robot_root, width = 640, height = 480)

## Mesh and Named Defaults
Not all models are created from primitives. Here, we will illustrate how to use meshes to create a model.

In [ ]:
realsense_d435i_assets = 'mujoco_menagerie/realsense_d435i/assets/'

In [ ]:
def realsense_model(path:str):
  spec = mj.MjSpec()
  spec.compiler.degree = False
  root = spec.worldbody

  # defaults
  main = spec.default()
  main.material.specular = 0
  main.material.shininess = 0.25

  # visual default
  v_def = spec.add_default('visual', main)
  v_def.geom.group = 2
  v_def.geom.type = mj.mjtGeom.mjGEOM_MESH
  v_def.geom.contype = 0
  v_def.geom.conaffinity = 0
  v_def.geom.mass = 0

  # collision default
  c_def = spec.add_default('collision', main)
  c_def.geom.group =3
  c_def.geom.type = mj.mjtGeom.mjGEOM_MESH
  c_def.geom.mass = 0

  materials = {
    "Black_Acrylic":[0.070360, 0.070360, 0.070360, 1],
    "Cameras_Gray":[0.296138, 0.296138, 0.296138, 1],
    "IR_Emitter_Lens":[0.287440, 0.665387, 0.327778, 1],
    "IR_Lens":[0.035601, 0.035601, 0.035601, 1],
    "IR_Rim":[0.799102, 0.806952, 0.799103, 1],
    "Metal_Casing":[1, 1, 1, 1],
    "RGB_Pupil":[0.087140, 0.002866, 0.009346, 1]
  }
  for name,rgba in materials.items():
    spec.add_material(name=name,rgba=rgba)

  for i in range(9):
    spec.add_mesh(name="d435i_"+str(i),
                  file=path+"d435i_"+str(i)+".obj")
  body = root.add_body(name="d435i")

  #visual geoms
  materials_order = ["IR_Lens","IR_Emitter_Lens",
                     "IR_Rim","IR_Lens","Cameras_Gray","Black_Acrylic",
                     "Black_Acrylic","RGB_Pupil"
                     ]
  for i,material in enumerate(materials_order):
    body.add_geom(default = v_def,
                  meshname="d435i_"+str(i),
                  material=material
                  )
  body.add_geom(
    default = v_def,
    meshname="d435i_8",
    material="Metal_Casing",
    mass=0.072
  )
  # collision geom
  body.add_geom(default = c_def,
                type=mj.mjtGeom.mjGEOM_CAPSULE,
                meshname="d435i_8"
                )

  return root, spec

In [ ]:

robot_root, spec = realsense_model(realsense_d435i_assets)
model = spec.compile()
render(model)

## Camera

In [ ]:
def camera_model(path:str):
  camera_shell, spec = realsense_model(path)

  camera_name = "realsense"
  camera_shell.add_camera(name = camera_name,
                          mode = mj.mjtCamLight.mjCAMLIGHT_FIXED ,
                           pos= [0,0,0],
                          euler = [3.14,0,0])
  return camera_shell, spec, camera_name


In [ ]:
robot_root, spec,camera_name = camera_model(realsense_d435i_assets)
model = spec.compile()
data = mj.MjData(model)

scene_option = mj.MjvOption()
scene_option.flags[mj.mjtVisFlag.mjVIS_CAMERA] = True
print(f"camera name::{camera_name}")
with mj.Renderer(model, 480, 640) as renderer:
  mj.mj_forward(model, data)
  renderer.update_scene(data, scene_option = scene_option)
  media.show_image(renderer.render())

## Height field
Height fields are used to represent uneven terrain or mountains. There are many ways to generate procedural terrain. Here, we will use [Perlin Noise](https://www.youtube.com/watch?v=9x6NvGkxXhU) to generate a height field.

For more inspiration, have a look at this [video](https://www.youtube.com/watch?v=gsJHzBTPG0Y).

We start by installing a [Python package](https://github.com/pvigier/perlin-numpy) that helps with the generation of Perlin Noise.

In [ ]:
# perlin noise
!pip install git+https://github.com/pvigier/perlin-numpy
from perlin_numpy import generate_perlin_noise_2d

In [ ]:
np.random.seed(0)
noise = generate_perlin_noise_2d((256, 256), (8, 8))
plt.imshow(noise, cmap='gray', interpolation='lanczos')
plt.colorbar()

In [ ]:

class ArenaHFileld:
  def __init__(self):
    self.spec = mj.MjSpec()
    self.root = self.spec.worldbody
    self.spec.compiler.degree = False
    self.scene_option = mj.MjvOption()

    # adding hfield
    noise = generate_perlin_noise_2d((128, 128), (8, 8))
    hfield = self.spec.add_hfield(name ='hfield',
                                  size = [10,10,1,1],
                                  nrow = noise.shape[0],
                                  ncol = noise.shape[1],
                                  userdata = noise.flatten())
    # Make arena with textured floor.
    chequered = self.spec.add_texture(
        name = "chequered", type=mj.mjtTexture.mjTEXTURE_2D,
        builtin = mj.mjtBuiltin.mjBUILTIN_CHECKER,
        width = 300, height = 300, rgb1 = [.2, .3, .4],
        rgb2 = [.3, .4, .5])

    grid = self.spec.add_material(
        name = 'chequered', texrepeat = [5, 5], reflectance = .2
        ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'chequered'
    self.root.add_geom(
        type = mj.mjtGeom.mjGEOM_HFIELD,
        material = 'chequered', hfieldname = 'hfield')

    for x in [-5, 0, 5]:
      for y in [-5, 0, 5]:
        self.root.add_light(pos = [x, y, 3], dir = [-x, 1, -2])

    self.camera = self.root.add_camera(name = "topview",
                          mode = mj.mjtCamLight.mjCAMLIGHT_FIXED ,
                           pos= [0,0,15],
                          euler = [0,0,0])



  def add_movable_asset(self, asset , pos = [0,0,0], prefix = '_'):
    frame = self.root.add_frame(pos = pos)
    body = frame.attach_body(asset, prefix, '')
    body.add_freejoint()
    return body


In [ ]:
arena = ArenaHFileld()
q_model, _ = quadruped_model()

# robots
for x in np.arange(-5,5,2.5):
  for y in np.arange(-5,5,2.5):
      arena.add_movable_asset(q_model,[x,y,1],str(x)+str(y))

model = arena.spec.compile()
data = mj.MjData(model)
# print_xml(arena.spec.to_xml())

# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
amp = 0.9

actuators = [data.bind(actuator) for actuator in arena.spec.actuators]

# Simulate and display video.
duration = 2    # (seconds)
framerate = 30  # (Hz)
frames = []

mj.mj_resetData(model, data)
with mj.Renderer(model,width=640,height=480) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    for i, actuator in enumerate(actuators):
      actuator.ctrl = amp * np.sin(freq * data.time + phase[i])
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      # renderer.update_scene(data,scene_option=arena.scene_option)
      renderer.update_scene(data,camera="topview")

      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

## Procedural Mesh Generation
Mujoco has the functionality to create a mesh given vertices. These vertices can be the result of procedural mesh generation. A person can use libraries like [CadQuery](https://cadquery.readthedocs.io/en/latest/) to generate meshes and then load them with Mujoco. Here, we will not demonstrate procedural mesh generation; however, we will see how the resulting vertices can be used in Mujoco.

If you would like to create your own low-poly rock, have a look at this [video](https://www.youtube.com/watch?v=1H2YaRPfpdM).

### Low Poly rock

In [ ]:
!pip install pyvista

import pyvista as pv
import numpy as np


In [ ]:
def low_poly_rock_mesh(perturbation = 0.2):
  # low poly sphere
  sphere =  pv.Sphere(radius=1.0, theta_resolution=5, phi_resolution=5)

  # scaling to make intial shape more rock like
  scale=[1.0, 0.2, 0.5]
  sphere = sphere.scale(scale)

  # get sphere vertices
  points = sphere.points

  # perturbe each vertex within perturbation range
  perturbations = np.random.uniform(-perturbation,
                                    perturbation,
                                    size=points.shape)

  perturbed_points = points + perturbations
  sphere.points = perturbed_points

  return sphere

def sample_points():
  samples = np.random.normal(loc=0, scale= 0.1,size = (100,3))

  return samples



### High resolusion rock

In [ ]:
!pip install git+https://github.com/mohammad200h/mj_rock_generator.git
from mj_rock_generator import rock
import tempfile
from PIL import Image

### Rock Mjspec Model

In [ ]:
def rock_model(texture,rock_func,smooth_rock = False, name="rock", scale = 1):
  spec = mj.MjSpec()
  root = spec.worldbody
  spec.compiler.degree = False

  main = spec.default()
  main.mesh.scale = [random.random()*scale] * 3

  spec.add_texture(
        name="rock", type=mj.mjtTexture.mjTEXTURE_2D,
        file = texture,
        width=300, height=300)

  spec.add_material(
        name='rock',  reflectance=.2
        ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'rock'
  # defaults
  main = spec.default()
  main.geom.type = mj.mjtGeom.mjGEOM_MESH

  mesh = rock_func()

  spec.add_mesh(
      name = "rock",
      # uservert = np.array(mesh.points).flatten()
      uservert = mesh.flatten()

  )
  root.add_geom(meshname = name )

  return root, spec

In [ ]:
arena = Arena()
low_ploy = True
rock_func = None

if low_ploy:
  rock_func = low_poly_rock_mesh
else:
  rock_func = rock

rock_func = sample_points
texture = generate_perlin_noise_2d((256, 256), (8, 8))

with tempfile.NamedTemporaryFile(suffix=".png") as temp_file:
  temp_file_path = temp_file.name
  # "L" mode for grayscale
  pil_image = Image.fromarray(texture, mode="L")
  pil_image.save(temp_file_path, format="png")

  r,_ = rock_model(temp_file_path,rock_func,smooth_rock = True)
  arena.add_movable_asset(r,[0,0,0.23])

  model = arena.spec.compile()
  render(model)

### Rocky Situation
Here, we will create a scene using almost every model we |have created. Then, we will add more obstacles to the scene by procedurally creating two layers of rocks. Finally, we will create a robot called Frank and place it in the middle of the rocks.

### Meet Frank:
We will use everything we have learned to create a robot called Frank. Frank's lower body is made of a quadruped. His upper body is made of a hexapod, keeping his arms level while moving. Frank's shorter arm is made of spatial tendons. In contrast, his longer arm is made of a fixed tendon with an end effector capable of adhesion.

In [ ]:
def frank(realsense_d435i_assets:str):

  quadroped,quadroped_spec = quadruped_model()
  hexopad,_, top = make_hexapod(equality2 = True,
                 exclude_contact = True)

  # sticky arm
  sticky_arm, leaf, spec = arm_model_fixed_tendon()

  ee_body = leaf.add_body(pos=[0,0,0.25],name="ee")
  ee_body.add_geom(
      type = mj.mjtGeom.mjGEOM_SPHERE,
      size = [0.05,0,0],
      mass = 0.1,
      rgba = [1,1,0,1]
  )

  # TODO: Ask Alessio if this is correct
  spec.add_actuator(
      name = "act_glue",
      ctrllimited = 1,
      gaintype = mj.mjtGain.mjGAIN_FIXED,
      biastype = mj.mjtBias.mjBIAS_NONE,
      trntype = mj.mjtTrn.mjTRN_BODY
  )

  # camera
  camera, _, camera_name = camera_model(realsense_d435i_assets)
  cam_frame = top.add_frame(pos=[0,0,0.03],euler=[0,0,1.57])
  cam_frame.attach_body(camera, "","_")

  #
  muscle_arm, _, _ = arm_model_muscle()

  arm_frames = [
      {
          "name":"left",
          "pos":[0.1,0,0],
          "euler":[0,1.2,0],
          "arm":sticky_arm
      },
      {
          "name":"right",
          "pos":[-0.1,0,0],
          "euler":[0,-1.2,0],
          "arm":muscle_arm
      }
  ]

  body = top

  for prop in arm_frames:
    frame = body.add_frame(pos = prop['pos'],euler=prop['euler'])
    frame.attach_body(prop['arm'],"","_"+prop["name"])


  frame = quadroped.add_frame(pos=[0,0,0])
  frame.attach_body(hexopad, '','_')


  return quadroped ,quadroped_spec,camera_name


In [ ]:
arena = Arena()
robot_root, _, _ = frank(realsense_d435i_assets)
arena.add_fixed_asset(robot_root,[0,0,0.15])
model = arena.spec.compile()
render(model,height = 400)

In [ ]:
def circle_coordinates(radius, angle):
    x = radius * math.cos(angle)
    y = radius * math.sin(angle)
    return x, y


In [ ]:
arena = Arena()
robot, _ , realsense = frank(realsense_d435i_assets)
random_state = np.random.RandomState(42)

realsense = "_realsense__right_"

# rock properties
texture = generate_perlin_noise_2d((256, 256), (8, 8))
radius = 1
scale  = 1
rock_func = sample_points

with tempfile.NamedTemporaryFile(suffix=".png") as temp_file:
  temp_file_path = temp_file.name
  # "L" mode for grayscale
  pil_image = Image.fromarray(texture, mode="L")
  pil_image.save(temp_file_path, format="png")

  # inner rocks
  for i,angle in enumerate(np.linspace(-3.14,3.14,10)):
    x,y = circle_coordinates(radius/2,angle)
    name = "inner_rock_"+str(i)

    asset,_ = rock_model(temp_file_path,rock_func, scale = scale)
    arena.add_movable_asset(asset,[x,y,0.11],prefix=name)

  # outer rocks
  for i,angle in enumerate(np.linspace(-3.14,3.14,20)):
    x,y = circle_coordinates(radius,angle)
    name = "outer_rock_"+str(i)

    asset,_ = rock_model(temp_file_path,rock_func, scale = scale)
    arena.add_movable_asset(asset,[x,y,0.11],prefix=name)

  # robot
  arena.add_movable_asset(robot,[0,0,3])
  top_camera = arena.add_tracking_camera("_world",euler=[0,0,0],pos=[0,0,3])
  model = arena.spec.compile()
  data = mj.MjData(model)



  # print(arena.spec.to_xml())
  actuators = [data.bind(actuator) for actuator in arena.spec.actuators]
  # Control signal frequency, phase, amplitude.
  freq = 5
  phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
  amp = 0.9

  # Simulate and display video.
  duration = 5    # (seconds)
  framerate = 30  # (Hz)
  frames = []
  realsense_frames = []

  mj.mj_resetData(model, data)
  with mj.Renderer(model,width=640,height=480) as renderer:
    while data.time < duration:
      # Inject controls and step the physics.
      for i, actuator in enumerate(actuators):
        actuator.ctrl = amp * np.sin(freq * data.time + phase[i])
      mj.mj_step(model, data)
      if len(frames) < data.time * framerate:
        # top camera
        renderer.update_scene(data,camera=top_camera)
        pixels = renderer.render()
        frames.append(pixels)
        # realsense camera
        renderer.update_scene(data,camera=realsense)
        pixels = renderer.render()
        realsense_frames.append(pixels)

  media.show_video(frames, fps=framerate)
  print("\n realsense camera view: \n")
  media.show_video(realsense_frames, fps=framerate)
